In [140]:
import numpy as np
import cvxpy as cp
import itertools
from itertools import product as iters

I. TWO POINT QUANTUM CORRELATORS

Direct implementation of what is in the exercises

In [141]:
def B(P, n):
    s1 = 0
    for i in range(0, n): #from 1 to n in the sheet
        s1 += exAB(i, i, P, n)
    s2 = 0
    for i in range(0, n-1): #from 1 to n-1 in the sheet
        s2 += exAB(i+1, i, P, n)
    se = exAB(0, n-1, P, n)
    s = s1+s2-se
    return s

def exAB(x, y, P, n):
    S = 0
    for a, b in iters([0, 1], repeat=2):
            S += (-1)**(a+b)*P[1*a+2*b, 1*x+n*y]
    return S

In [142]:
for n in range(2, 7):
    P = cp.Variable((4, n**2))
    constraints = [P >= 0]
    constraints+= [P <= 1]
    constraints+= [cp.sum(P) == 2*n]
    for i in range(0, 4):
        constraints+= [cp.sum(P[i, :]) == 1]
    for j in range(0, n):
        constraints+= [ cp.sum(P[:, j]) == 2/n]
    prob = cp.Problem(cp.Maximize(B(P, n)), constraints)
    print("n = " + str(n) + " |", prob.solve())

n = 2 | 3.999999998627687
n = 3 | -inf
n = 4 | -inf
n = 5 | -inf
n = 6 | -inf


Different approach based on lecture notes

Main idea:
Define γ = (
    0, Pt
    P, 0 
)
Define P based on the functional
Do X = cp.Variable((2*n, 2*n), symmetric=True)
cp.Optimize(cp.trace(γ@X))

In [143]:
def Pf(n):
    P = np.zeros((n, n))
    for i in range(0, n):
        P[i, i] = 1
    for i in range(0, n-1):
        P[i+1, i] = 1
    P[0, n-1] = -1
    return P

def gamma(P,n):
    zer = np.zeros((n,n))
    g = np.block([[zer, P.T], [P, zer]])
    return g

In [144]:
for n in range(2, 7):

    P = Pf(n)
    g = gamma(P,n)


    X = cp.Variable((2*n, 2*n), PSD=True)
    
    con = []
    con+= [X[i, i] == 1 for i in range(0, 2*n)]
    #for i in range(2*n):
    #    con.append(X[i, i] == 1)

    obj = cp.Maximize( cp.trace(g@X)/2 )
    prob = cp.Problem( obj , con)
    print("n = " + str(n) + " |", prob.solve())


n = 2 | 2.828427124639405
n = 3 | 5.196152427563769
n = 4 | 7.391037008387376
n = 5 | 9.51059333335389
n = 6 | 11.591107813376679


II. INFORMATION CAUSALITY

In [145]:
import sympy as sy
from sympy.abc import a,b,x,y,l
from sympy import DiracDelta as DD
x0 = sy.Symbol("x_0")
x1 = sy.Symbol("x_1")

We want to see if information causality is violated, or better said for what λ it is. 

That means we to find the λ for which the following equation is violated/does not hold:
$$ \sum_y I(b:X_y) \leq m = 1$$
we use that
$$ I(b:X_y) = H(b) + H(X_y) - H(b, X_y)$$
where $H(b, X_y)$ is the joint entropy.
We can also split the term $I(b:X_y) = I(b:X_0) + I(b:X_1)$ and calculate.

Generally:
$$ H(x) = -P(x) log_2(P(x)) $$
which can be done analogously for $H(b, X_y)$,

$$ H(b, X_y) = -\sum_{b,y} P(b, X_y) \log_2(P(b, X_y)) $$

Secondary note:
$$ H(X|Y):=\sum_{xy} p(x|y)\log p(x|y) p(y) $$

To clarify:
$$ H(b) $$
takes the isotropic distribution.

While 
$$ H(X_y)$$
takes a uniform distribution, i.e equal outcomes for any input. We have $x_0, x_1 \in \{ 0, 1\}$.

In [195]:
#Dirac delta works the same way a kronecker delta
def Piso(a, b, x, y, l):
    Pp = DD( ((a+b)%2)   - x*y)
    Pn = DD( ((a+b+1)%2) - x*y )
    return (l*Pp + (1 - l)*Pn)/2

In [198]:
#run a test
Piso(a,b,x,y,l).subs(DD(0), 1)

l*DiracDelta(x)/2 + (1 - l)*DiracDelta(1 - x)/2

In [200]:
#Some testing, on how to get rid of δ(0)
Piso(0,0,1,0,l), Piso(0,0,1,0,l).subs(DD(0), 1)

(l*DiracDelta(0)/2, l/2)

In [202]:
#a uniform distribution for 2 possible inputs
def P_uni2():
    return 1/2

In [201]:
#Lets define some entropies
#Where f is a probabity distribution function P(X) for example.
def H(f):
    return -f*sy.log(f, 2)

#for 2 variables? might not be needed
#H(X,Y) = - Σ_{x,y} p(x,y) * log(p(x,y))

In [203]:
#So for H(X_y) we have:
#X_0, can take 0,1 as likely
HX0 = np.sum(np.array([H(P_uni2()) for n in range(0,2)]))
HX0 = HX0.evalf()
#obviously the result will be 1, but it is good to see the functions work
HX1 = HX0
print(HX0, HX1)

1.00000000000000 1.00000000000000


Now we want to find out what $ H(b) $ is, which is based off of $P_{iso}(a, b, x, y, l)$. 

$P(b = 0| y = 0)$, there are particular permutations of $ a \oplus \tilde{b} \oplus x_0 $.

$P(a \oplus \tilde{b} \oplus x_0| y = 0) = P(a, b | x_0, y=0) \cdot P(x_0|y=0)$

Where the 2nd term in the product is uniformly distributed. 

The first term is obtain from the isotropic distribution.

In [229]:
#We have y = 0, b = 0/1 
#Pby
y=0
P00 = 0
P10 = 0
for a, b, x0 in iters([0, 1], repeat =3):
    if a ^ b ^ x0 == 0: # we look at h
        P00+= Piso(a, b, x0, y,l)/2
    elif a ^ b ^ x0 == 1:
        P10+= Piso(a, b, x0, y,l)/2
P00 = P00.subs(DD(0), 1)
P10 = P10.subs(DD(0), 1)
print(P00, P10)

1/2 1/2


In [230]:
#We have y = 1, b = 0/1 
#Pby
y=1
P01 = 0
P11 = 0
for a, b, x0 in iters([0, 1], repeat =3):
    if a ^ b ^ x0 == 0: # we look at h
        P01+= Piso(a, b, x0, y,l)/2
    elif a ^ b ^ x0 == 1:
        P11+= Piso(a, b, x0, y,l)/2
P01 = P01.subs(DD(0), 1)
P11 = P11.subs(DD(0), 1)
print(P00, P10)

1/2 1/2


So we obtain $P(b|y) = 1/2$ for all permutation of b,y.

Finally we have to figure out how to calculate $H(b,X_y)$, which is based on the distribution $P(b, X_y)$.

Generally: $P(a,b) = P(a) \cdot P(b|a)$